Essa é uma forma de verificar como está indo as ações

In [6]:
from binance.client import Client
from secrets import api_secrets, api_key
from binance.enums import *
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from scipy import stats
from time import sleep

client = Client(api_key, api_secrets)
print(client.get_account_status())
MOEDA = 'SANTOSBUSD'
initial_price = float(client.get_recent_trades(symbol=MOEDA)[-1]['price'])
M = 30_000
TX = 0.001
brl_i = M/(2+TX)
chz_i = ((M-brl_i)*(1-TX))/initial_price
xi = initial_price

{'data': 'Normal'}


In [3]:
def p_delta():    
    delta_x = xf - xi
    porc_delta = (delta_x/xi)*100
    return porc_delta

def transf(brl, chz, xf):
    def calc_lucro(brl, chz, xf):    
        return chz*xf - brl
    l = calc_lucro(brl, chz, xf)
    return abs(l/(2-TX))

def od_compra(brl, chz, xf):
    t = transf(brl, chz, xf)   
    brl_f = brl - t
    chz_f = chz + (t/xf)*(1-TX)
    return brl_f, chz_f

def od_venda(brl, chz, xf):
    t = transf(brl, chz, xf)  
    brl_f = brl + t*(1-TX)
    chz_f = chz - (t/xf)
    return brl_f, chz_f

def s_venda(brl, chz, xf, xi):
    brl_t, chz_t = od_venda(brl, chz, xf)
    brl_f, chz_f = od_compra(brl_t, chz_t, xi)
    return brl_f, chz_f

def s_compra(brl, chz, xf, xi):
    brl_t, chz_t = od_compra(brl, chz, xf)
    brl_f, chz_f = od_venda(brl_t, chz_t, xi)
    return brl_f, chz_f


In [5]:
while True:
    xf = float(client.get_recent_trades(symbol=MOEDA)[-1]['price'])
    print(xf, xi, initial_price)

    brl_f, chz_f = s_venda(brl_i, chz_i, xf, xi)
    delta_brl = brl_f - brl_i
    delta_chz = chz_f - chz_i
    print(delta_brl, delta_chz)

    brl_f, chz_f = s_compra(brl_i, chz_i, xf, xi)
    delta_brl = brl_f - brl_i
    delta_chz = chz_f - chz_i
    print(delta_brl, delta_chz)

    print(100*((xf - xi)/xi))
    sleep(10)

4.527 4.526 4.526
-0.008967307112470735 0.0013312409719219431
-0.009324014472440467 0.0012524280336947413
0.02209456473708206


In [57]:
x = 1_000_000 * 0.04
x

40000.0